In [1]:
!pip install opencv-python


In [2]:
import cv2
import numpy as np
from google.colab import files


In [3]:
input_video = "20s.mp4"
output_video = "sobel_preprocessed_fish.mp4"

cap = cv2.VideoCapture(input_video)

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))


In [4]:
def sobel_edges(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

    magnitude = cv2.magnitude(sobel_x, sobel_y)
    magnitude = np.uint8(np.clip(magnitude, 0, 255))

    return magnitude


In [5]:
def blend_sobel_rgb(frame, sobel):
    sobel_bgr = cv2.cvtColor(sobel, cv2.COLOR_GRAY2BGR)
    blended = cv2.addWeighted(frame, 0.7, sobel_bgr, 0.3, 0)
    return blended


In [6]:
def replace_blue_channel(frame, sobel):
    b, g, r = cv2.split(frame)
    b = sobel  # replace blue channel
    merged = cv2.merge((b, g, r))
    return merged


In [7]:
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. Sobel edge detection
    sobel = sobel_edges(frame)

    # 2. Blend Sobel with RGB
    blended = blend_sobel_rgb(frame, sobel)

    # 3. Replace blue channel
    final_frame = replace_blue_channel(blended, sobel)

    out.write(final_frame)

    frame_count += 1
    if frame_count % 50 == 0:
        print(f"Processed {frame_count} frames")


Processed 50 frames
Processed 100 frames
Processed 150 frames
Processed 200 frames
Processed 250 frames
Processed 300 frames
Processed 350 frames
Processed 400 frames
Processed 450 frames
Processed 500 frames
Processed 550 frames
Processed 600 frames


In [8]:
cap.release()
out.release()
cv2.destroyAllWindows()

print("Preprocessed video saved as:", output_video)


Preprocessed video saved as: sobel_preprocessed_fish.mp4


In [9]:
files.download("sobel_preprocessed_fish.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>